In [300]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.nn.functional as F

In [301]:
sample_rate = 1
el_len = 50
n_fold = 10

act_map = {'work':0, 
           'eating':1, 
           'toilet':2, 
           'fitness':3, 
           'sleep':4, 
           'personal_hygiene':5, 
           'shower':6, 
           'relax':7, 
           'cooking':8, 
           'phonecall':9, 
           'leave_home':10, 
           'null':11}

# MTS data loading and its transformation to (X, Y) pairs

In [302]:
data_df = pd.read_csv('act_mts_data.csv')
data_df.head()

,Unnamed: 0,time,snd11_roadside,snd12_bed,snd21_kitchen,snd22_desk,tmp1,hmd1,light1,mot1,tmp2,hmd2,light2,mot2,weekend,hour,null-1,null-2,act
0,0,2020-05-07 22:19:54.111575-04:00,0.112980,0.090066,0.125199,0.115151,0.579129,0.300179,0.06409,0.249023,0.585029,0.302947,0.03906,0.239700,0,0.916667,0,0,0
1,1,2020-05-07 22:19:57.473695-04:00,0.113832,0.090506,0.125769,0.127575,0.579029,0.300684,0.06415,0.248947,0.585029,0.302947,0.03903,0.244583,0,0.916667,0,0,0
2,2,2020-05-07 22:19:58.590782-04:00,0.114485,0.089217,0.118430,0.119727,0.579129,0.300684,0.06412,0.247681,0.585029,0.302568,0.03906,0.250595,0,0.916667,0,0,0
3,3,2020-05-07 22:20:00.861969-04:00,0.113706,0.101555,0.122844,0.126247,0.578929,0.300926,0.06415,0.248154,0.585129,0.302705,0.03906,0.242569,0,0.916667,0,0,0
4,4,2020-05-07 22:20:06.581213-04:00,0.115635,0.082554,0.133055,0.122825,0.579029,0.300926,0.06409,0.248978,0.585029,0.302389,0.03903,0.248520,0,0.916667,0,0,0


In [303]:
subset_x = data_df[['snd11_roadside', 'snd12_bed', 'snd21_kitchen', 'snd22_desk', 'tmp1', 'hmd1', 'light1', 'mot1', 'tmp2', 'hmd2', 'light2', 'mot2', 'weekend', 'hour', 'null-1', 'null-2']]
subset_y = data_df['act']
dataset_features = subset_x.values.tolist()
dataset_labels = subset_y.tolist()

type(dataset_labels), len(dataset_labels), type(dataset_features), len(dataset_features)

(list, 203531, list, 203531)

## Reshaping input features to 50x4x4 (C x H x W)
* To make make an input feqture of shape (50x4x4) after ToTensor transformation later, 4x4x50 matrix should be created here.
* By adjusting the data sampling rate, each data element can cover longer period of time (e.g., 1/2 data sampling results in 2 mins of MTS data). 
* The majority voting can be used to decide each cubic data point's label.

In [304]:
features_arr = np.asarray(dataset_features)
labels_arr = np.asarray(dataset_labels)

residual = len(dataset_labels)%el_len

features_arr = features_arr[:-residual]
labels_arr = labels_arr[:-residual]

features_arr.shape, labels_arr.shape, type(features_arr[0]), len(features_arr[0])
#labels_arr

((203500, 16), (203500,), numpy.ndarray, 16)

### Reshaping feature array to 50x4x4 array

In [305]:
features_arr = features_arr.reshape(int(len(features_arr)/el_len), el_len, 4, 4)
features_arr = np.transpose(features_arr, (0,2,3,1))
features_arr.shape

(4070, 4, 4, 50)

### Squeezing labels by means of the most frequent values in each 50-length array

In [306]:
labels_arr = labels_arr.reshape(int(len(labels_arr)/el_len), el_len)
labels_arr = [np.bincount(x).argmax() for x in labels_arr]
labels_arr = np.asarray(labels_arr)
labels_arr, labels_arr.shape

(array([0, 0, 0, ..., 0, 0, 0]), (4070,))

In [307]:
dataset = tuple(zip(features_arr, labels_arr))
dataset = np.asarray(dataset)
dataset.shape, dataset[0].shape, dataset[0][0].shape, dataset[0][1].shape

((4070, 2), (2,), (4, 4, 50), ())

## Divide the dataset into training and validation datasets
Considering the characteristics of human activities in a smart home, splitting the dataset in an arbitrary basis may not be an optimal choice with respect to the recognition accuracy. However, the main goal of this experiment is to see if n-minute-long MTS data can represent any meaningful patterns of human activities. Therefore, we split the dataset arbitrarily in this notebook.

In [308]:
n_data = dataset.shape[0]
n_val = int(n_data * 0.1)

shuffled_indices = torch.randperm(n_data)
train_indices = shuffled_indices[:-n_val]
val_indices = shuffled_indices[-n_val:]

dataset_train = dataset[train_indices]
dataset_val = dataset[val_indices]

dataset_train.shape, dataset_val.shape

((3663, 2), (407, 2))

## Transform data to tensor
By definition, **ToTensor** transforms the image or numpy.ndarray to a tensor with range [0,1]. As the result, the input shape **(H x W x C)** is transformed into the output shape **(C x H x W)** in the range of [0, 1].

In [309]:
to_tensor = transforms.ToTensor()

dataset_train_t = [(to_tensor(mts).to(torch.float32), label) for mts, label in dataset_train]
dataset_val_t = [(to_tensor(mts).to(torch.float32), label) for mts, label in dataset_val]

# Building a simple PyTorch CNN model

## Deep Learning model training and validation

In [311]:
train_loader = torch.utils.data.DataLoader(dataset_train_t, batch_size = 64, shuffle=True)
val_loader = torch.utils.data.DataLoader(dataset_val_t, batch_size = 64, shuffle=True)

In [312]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_loader):
    for epoch in range(1, n_epochs+1):
        loss_train = 0.0
        for imgs, labels in train_loader:
            outputs = model(imgs)
            loss = loss_fn(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            loss_train += loss.item()
            
        if epoch == 1 or epoch % 10 == 0:
            print('{} Epoch {}, Training loss {}'.format(datetime.now(), epoch, float(loss_train)))

In [313]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.n_channel = el_len
        self.conv1 = nn.Conv2d(self.n_channel, self.n_channel*3, kernel_size=3, padding=1) # IN: 50x4x4, OUT: 150x4x4
        self.conv1_bn = nn.BatchNorm2d(self.n_channel*3)
        # maxpool2d, IN: 50x4x4, OUT: 50x2x2
        self.conv2 = nn.Conv2d(self.n_channel*3, self.n_channel*2, kernel_size=2, padding=1) # IN: 150x2x2, OUT: 100x1x1
        self.conv2_bn = nn.BatchNorm2d(self.n_channel*2)
        # maxpool2d, IN: 50x3x3, OUT: 50x1x1
        self.fc1 = nn.Linear(1*1*self.n_channel*2, 32)
        self.fc2 = nn.Linear(32,len(act_map.keys()))
        
    def forward(self, x):
        out = F.max_pool2d(F.relu(self.conv1_bn(self.conv1(x))), 2)
        out = F.max_pool2d(F.relu(self.conv2_bn(self.conv2(out))), 2)
        out = out.view(-1, 1*1*self.n_channel*2) 
        out = F.relu(self.fc1(out))
        out = self.fc2(out)
        return out

In [ ]:
model = Net()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
loss_fn = nn.CrossEntropyLoss()

training_loop(n_epochs = 250, optimizer = optimizer, model = model.train(), loss_fn = loss_fn, train_loader = train_loader)

2020-05-13 22:26:37.118826 Epoch 1, Training loss 99.1751571893692
2020-05-13 22:26:47.102250 Epoch 10, Training loss 0.48210770823061466
2020-05-13 22:26:58.131146 Epoch 20, Training loss 0.18653730326332152
2020-05-13 22:27:09.522935 Epoch 30, Training loss 0.11183002323377877
2020-05-13 22:27:21.128023 Epoch 40, Training loss 0.07862311461940408
2020-05-13 22:27:33.415424 Epoch 50, Training loss 0.05938897019950673
2020-05-13 22:27:45.833668 Epoch 60, Training loss 0.04791862261481583
2020-05-13 22:27:59.086499 Epoch 70, Training loss 0.03998408810002729
2020-05-13 22:28:11.567491 Epoch 80, Training loss 0.03413496620487422
2020-05-13 22:28:24.417799 Epoch 90, Training loss 0.02972908320953138
2020-05-13 22:28:37.106330 Epoch 100, Training loss 0.026218993036309257
2020-05-13 22:28:49.811969 Epoch 110, Training loss 0.02350875682896003
2020-05-13 22:29:03.034967 Epoch 120, Training loss 0.02120962250046432
2020-05-13 22:29:16.300005 Epoch 130, Training loss 0.019386974279768765
2020

In [261]:
train_loader = torch.utils.data.DataLoader(dataset_train_t, batch_size = 64, shuffle=False)
val_loader = torch.utils.data.DataLoader(dataset_val_t, batch_size = 64, shuffle=False)

model = model.eval()

for loader in [train_loader, val_loader]:
    correct = 0
    total = 0
    
    with torch.no_grad():
        for imgs, labels in loader:
            outputs = model(imgs)
            _, predicted = torch.max(outputs, dim=1)
            total += labels.shape[0]
            correct += int((predicted == labels).sum())
    
    print("Accuracy: %f" % (correct / total))

Accuracy: 0.796791
Accuracy: 0.695652


# Visualize the answers and Compare them with ground truth